# Multi-Agent Conversation and Stand-up Comedy

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here it goes: 
Why did the math book look sad? Because it had too many problems.


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure! Please provide me with the joke you would like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! So I was at the supermarket the other day, and I saw a sign that said, "Baby carrots: Great for snacking!" And I thought, well, what about adult carrots? Are they just not good enough for snacking? Like, do they need to graduate to baby carrot status to be deemed snack-worthy?

--------------------------------------------------------------------------------
joe (to cathy):

That's like saying adult supervision is no longer needed because they graduated from baby supervision.

--------------------------------------------------------------------------------
cathy (to joe):

Nice one, Joe! It's true - just because you're an adult doesn't mean you can't benefit from a little supervision now and then. It's like, "Congratulations, you're an adult now! But we're still gonna need to double-check your decisions every o

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Sure thing, Joe! So I was at the supermarket the other day, and '
             'I saw a sign that said, "Baby carrots: Great for snacking!" And '
             'I thought, well, what about adult carrots? Are they just not '
             'good enough for snacking? Like, do they need to graduate to baby '
             'carrot status to be deemed snack-worthy?',
  'name': 'cathy',
  'role': 'user'},
 {'content': "That's like saying adult supervision is no longer needed because "
             'they graduated from baby supervision.',
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Nice one, Joe! It's true - just because you're an adult doesn't "
             "mean you can't benefit from a little supervision now and then. "
             'It\'s like, "Congratulations, you\'re an adult now! But we\'re '
             'still gonna need to double-check your decisions every on

In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 150,
                                                             'cost': 0.000376,
                                                             'prompt_tokens': 302,
                                                             'total_tokens': 452},
                                      'total_cost': 0.000376},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 150,
                                                             'cost': 0.000376,
                                                             'prompt_tokens': 302,
                                                             'total_tokens': 452},
                                      'total_cost': 0.000376}}


In [9]:
pprint.pprint(chat_result.summary)

("Nice one, Joe! It's true - just because you're an adult doesn't mean you "
 "can't benefit from a little supervision now and then. It's like, "
 '"Congratulations, you\'re an adult now! But we\'re still gonna need to '
 'double-check your decisions every once in a while."')


## Get a better summary of the conversation

In [10]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Sure thing, Joe! So I was at the supermarket the other day, and I saw a sign that said, "Baby carrots: Great for snacking!" And I thought, well, what about adult carrots? Are they just not good enough for snacking? Like, do they need to graduate to baby carrot status to be deemed snack-worthy?

--------------------------------------------------------------------------------
joe (to cathy):

That's like saying adult supervision is no longer needed because they graduated from baby supervision.

--------------------------------------------------------------------------------
cathy (to joe):

Nice one, Joe! It's true - just because you're an adult doesn't mean you can't benefit from a little supervision now and then. It's like, "Congratulations, you're an adult now! But we're still gonna need to double-check your decisions every o

In [11]:
pprint.pprint(chat_result.summary)

('Adults can still benefit from supervision and guidance, just like how adult '
 'carrots can still be good for snacking despite the focus on baby carrots.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, ready for some laughs? Let's do this!

--------------------------------------------------------------------------------
joe (to cathy):

Absolutely, Cathy! Let's bring on the humor. What's your favorite type of joke?

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go.

--------------------------------------------------------------------------------


In [14]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

We were just about to share some jokes, but you mentioned you had to go. Do you want to share some jokes before you go?

--------------------------------------------------------------------------------
cathy (to joe):

I gotta go.

--------------------------------------------------------------------------------
